<a href="https://colab.research.google.com/github/BhaveshWadibhasme/Advance-NLP/blob/main/refined_rag_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# Extract data file from web
#!wget https://www.rgpv.ac.in/campus/BTech_I/transformer.pdf

In [57]:
#!pip install langchain_community
#!pip install langchain_openai
#!pip install pypdf
#!pip install chromadb
#!pip install tiktoken

In [58]:
import pandas as pd
import numpy as np
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [59]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [60]:
# Read dataset using data loader
data_path = "/content/transformer.pdf"
loader = PyPDFLoader(data_path)
documents = loader.load()

In [61]:
# Split document into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)
doc_chunk = splitter.split_documents(documents)

In [67]:
# Convert documents to embeddings
db = Chroma.from_documents(doc_chunk,OpenAIEmbeddings())
# sample_query = "What are type of Transformers?"
# retrived_output = db.similarity_search(sample_query)
retriver = db.as_retriever(search_kwargs={"k": 5})

In [68]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are helpful assistance which refers to provided context to answer to user question.{context}"),
        ("user","{user_input}"),
    ]
)

In [69]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
  model = "gpt-4o-mini",
  temperature = 0,
  max_tokens = None,
  max_retries = None,
  timeout = None,
  )

In [70]:
llm_chain = (
    {"context":retriver,"user_input":RunnablePassthrough()}
    |prompt
    |llm
)

In [71]:
llm_chain.invoke("What are the types of transfoermers?").content

'Transformers can be classified based on different criteria, such as types of construction and types of cooling. Some common types of transformers include:\n\n1. **Power Transformers**: Used in transmission networks to step up or step down voltage levels.\n2. **Distribution Transformers**: Used to distribute electricity to consumers at lower voltage levels.\n3. **Instrument Transformers**: Used for measurement and protection purposes, including current transformers (CT) and voltage transformers (VT).\n4. **Isolation Transformers**: Used to isolate different sections of a power system for safety and noise reduction.\n5. **Auto Transformers**: A type of transformer with a single winding that acts as both the primary and secondary winding.\n6. **Three-Phase Transformers**: Used in three-phase power systems, which can be constructed as three single-phase transformers or as a single three-phase unit.\n\nThese classifications can further be divided based on their construction (e.g., core typ